Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Use FLAML to Tune OpenAI Models

FLAML offers a cost-effective hyperparameter optimization technique [EcoOptiGen](https://arxiv.org/abs/2303.04673) for tuning Large Language Models. Our study finds that tuning hyperparameters can significantly improve the utility of LLMs.

In this notebook, we tune OpenAI models for code generation. We use [the HumanEval benchmark](https://huggingface.co/datasets/openai_humaneval) released by OpenAI for synthesizing programs from docstrings. 

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option:
```bash
pip install flaml[openai]>=1.1.3
```

In [1]:
%pip install flaml[synapse]==1.2.1 xgboost==1.6.1 pandas==1.5.1 numpy==1.23.4 datasets --force-reinstall

StatementMeta(automl, 21, 7, Finished, Available)

  Cloning https://github.com/microsoft/FLAML.git to /tmp/pip-install-deiu1ftw/flaml_ef55daf853604154a69c151faab6be07
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/FLAML.git /tmp/pip-install-deiu1ftw/flaml_ef55daf853604154a69c151faab6be07
  Resolved https://github.com/microsoft/FLAML.git to commit ef5a17cd830cc996bb07e72d07a1d2f3fc0c74c6
  Preparing metadata (setup.py) ... - done
  Using cached xgboost-1.6.1-py3-none-manylinux2014_x86_64.whl (192.9 MB)
  Using cached pandas-1.5.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached numpy-1.23.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached lightgbm-3.3.5-py3-none

Set your OpenAI key:

In [2]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

StatementMeta(automl, 21, 9, Finished, Available)

If you use Azure OpenAI, uncomment the following:

In [3]:
# import openai
# openai.api_type = "azure"
# openai.api_base = "YOUR_API_ENDPOINT"
# openai.api_version = "2022-12-01"
# openai.api_key = os.getenv("OPENAI_API_KEY")

StatementMeta(automl, 21, 10, Finished, Available)

## Load dataset

First, we load the humaneval dataset. The dataset contains 164 examples. We use the first 20 for tuning the generation hyperparameters and the remaining for evaluation. In each example, the "prompt" is the prompt string for eliciting the code generation, "test" is the Python code for unit test for the example, and "entry_point" is the function name to be tested.

In [4]:
import datasets

seed = 41
data = datasets.load_dataset("openai_humaneval")["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "prompt": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data)
]
test_data = [
    {
        "prompt": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data, len(data))
]


StatementMeta(automl, 21, 11, Finished, Available)

Found cached dataset openai_humaneval (/home/trusted-service-user/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/trusted-service-user/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75/cache-1e8448101c1b32e8.arrow


Check a tuning example:

In [5]:
print(tune_data[1]["prompt"])

StatementMeta(automl, 21, 12, Finished, Available)


def compare(game,guess):
    """I think we all remember that feeling when the result of some long-awaited
    event is finally known. The feelings and thoughts you have at that moment are
    definitely worth noting down and comparing.
    Your task is to determine if a person correctly guessed the results of a number of matches.
    You are given two arrays of scores and guesses of equal length, where each index shows a match. 
    Return an array of the same length denoting how far off each guess was. If they have guessed correctly,
    the value is 0, and if not, the value is the absolute difference between the guess and the score.
    
    
    example:

    compare([1,2,3,4,5,1],[1,2,3,4,2,-2]) -> [0,0,0,0,3,3]
    compare([0,5,0,0,0,4],[4,1,1,0,0,-2]) -> [4,4,1,0,0,6]
    """



Here is one example of the unit test code for verifying the correctness of the generated code:

In [6]:
print(tune_data[1]["test"])

StatementMeta(automl, 21, 13, Finished, Available)

def check(candidate):

    # Check some simple cases
    assert candidate([1,2,3,4,5,1],[1,2,3,4,2,-2])==[0,0,0,0,3,3], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([0,0,0,0,0,0],[0,0,0,0,0,0])==[0,0,0,0,0,0], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3],[-1,-2,-3])==[2,4,6], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3,5],[-1,2,3,4])==[2,0,0,1], "This prints if this assert fails 1 (good for debugging!)"

    # Check some edge cases that are easy to work out by hand.
    assert True, "This prints if this assert fails 2 (also good for debugging!)"




## Define Success Metric

Before we start tuning, we need to define the success metric we want to opotimize. For each code generation task, if one of the returned responses can pass the test, we consider the task as successfully solved. Then we can define the mean success rate of a collection of tasks.

### Define a code executor

First, we write a simple code executor. The code executor takes the generated code and the test code as the input, and execute them with a timer.

In [7]:
import signal
import subprocess
import sys

def timeout_handler(signum, frame):
    raise TimeoutError("Timed out!")

signal.signal(signal.SIGALRM, timeout_handler)
max_exec_time = 3  # seconds

def execute_code(code):
    code = code.strip()
    with open("codetest.py", "w") as fout:
        fout.write(code)
    try:
        signal.alarm(max_exec_time)
        result = subprocess.run(
            [sys.executable, "codetest.py"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
        )
        signal.alarm(0)
    except TimeoutError:
        return 0
    return int(result.returncode == 0)

StatementMeta(automl, 21, 14, Finished, Available)

This function will create a temp file "codetest.py" and execute it in a separate process. It allows for 3 seconds to finish that code.

### Define a function to evaluate the success for a given program synthesis task

In [8]:
def success_metrics(responses, prompt, test, entry_point):
    """Check if the task is successful.

    Args:
        responses (list): The list of responses.
        prompt (str): The input prompt.
        test (str): The test code.
        entry_point (str): The name of the function.

    Returns:
        dict: The success metrics.
    """
    success_list = []
    n = len(responses)
    for i in range(n):
        response = responses[i]
        code = f"{prompt}{response}\n{test}\ncheck({entry_point})"
        succeed = execute_code(code)
        success_list.append(succeed)
    return {
        "expected_success": 1 - pow(1 - sum(success_list) / n, n),
        "success": any(s for s in success_list),
    }


StatementMeta(automl, 21, 15, Finished, Available)

## Use the tuning data to find a good configuration

### Import the oai and tune subpackages from flaml.

FLAML has provided an API for hyperparameter optimization of OpenAI models: `oai.Completion.tune` and to make a request with the tuned config: `oai.Completion.create`. First, we import oai from flaml:

In [9]:
from flaml import oai, tune

StatementMeta(automl, 21, 16, Finished, Available)

/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (urllib3 1.26.15 (/nfs4/pyenv-b224b66d-ff10-4da7-ad2a-97c4dedd98a2/lib/python3.8/site-packages), Requirement.parse('urllib3<=1.26.6,>=1.23')).


For (local) reproducibility and cost efficiency, we cache responses from OpenAI.

In [10]:
oai.Completion.set_cache(seed)

StatementMeta(automl, 21, 17, Finished, Available)

This will create a disk cache in ".cache/{seed}". You can change `cache_path` in `set_cache()`. The cache for different seeds are stored separately.

### Perform tuning

The tuning will take a while to finish, depending on the optimization budget. The tuning will be performed under the specified optimization budgets.

* `inference_budget` is the target average inference budget per instance in the benchmark. For example, 0.02 means the target inference budget is 0.02 dollars, which translates to 1000 tokens (input + output combined) if the text Davinci model is used.
* `optimization_budget` is the total budget allowed to perform the tuning. For example, 5 means 5 dollars are allowed in total, which translates to 250K tokens for the text Davinci model.
* `num_sumples` is the number of different hyperparameter configurations which is allowed to try. The tuning will stop after either num_samples trials or after optimization_budget dollars spent, whichever happens first. -1 means no hard restriction in the number of trials and the actual number is decided by `optimization_budget`.

Users can specify tuning data, optimization metric, optimization mode, evaluation function, search spaces etc.. The default search space is:

```python
default_search_space = {
    "model": tune.choice([
        "text-ada-001",
        "text-babbage-001",
        "text-davinci-003",
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 1)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{prompt}",
}
```

The default search space can be overridden by users' input.
For example, the following code specifies four choices for the prompt and a fixed list of stop sequences. For hyperparameters which don't appear in users' input, the default search space will be used. If you don't have access to gpt-4 or would like to modify the choice of models, you can provide a different search space for model.

In [12]:
config, analysis = oai.Completion.tune(
    data=tune_data,  # the data for tuning
    metric="expected_success",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=success_metrics,  # the evaluation function to return the success metrics
    # log_file_name="logs/humaneval.log",  # the log file name
    inference_budget=0.05,  # the inference budget (dollar)
    optimization_budget=3,  # the optimization budget (dollar)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=-1,
    model=tune.choice(
        [
            "text-davinci-002",
            "code-cushman-001",
        ]
    ),
    prompt=[
        "{prompt}",
        "# Python 3{prompt}",
        "Complete the following Python function:{prompt}",
        "Complete the following Python function while including necessary import statements inside the function:{prompt}",
    ],  # the prompt templates to choose from
    stop=["\nclass", "\ndef", "\nif", "\nprint"],  # the stop sequence
)


StatementMeta(automl, 21, 19, Finished, Available)

[I 2023-04-10 12:23:12,833] A new study created in memory with name: optuna
[I 2023-04-10 12:23:12,835] A new study created in memory with name: optuna


No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
[flaml.tune.tune: 04-10 12:23:12] {832} INFO - trial 1 config: {'prompt': 2, 'stop': 0, 'subspace': {'model': 'text-davinci-002', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 36}}
[flaml.tune.tune: 04-10 12:26:16] {215} INFO - result: {'expected_success': 0.7947565938469873, 'success': 0.8, 'total_cost': 1.0647, 'cost': 1.0647, 'inference_cost': 0.03736, 'training_iteration': 0, 'config': {'prompt': 2, '

### Output tuning results

After the tuning, we can print out the config and the result found by FLAML:

In [13]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

# save results to notebook_output.txt
from flaml.version import __version__ as flaml_version
import datetime
results = {"optimized config": config, "best result on tuning data": analysis.best_result,}
result_info_dict = {"result_name": "integrate_openai.ipynb + optimized config and best result on tuning data",
                    "flaml_version": flaml_version, 
                    "time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                     "results": results}
result_info = "result name: {result_name}, flaml version: {flaml_version}, time: {time}, results: {results}".format(**result_info_dict)
with open("notebook_output.txt", "a") as f:
    f.write("\n")
    f.write(result_info)

StatementMeta(automl, 21, 20, Finished, Available)

optimized config {'prompt': 'Complete the following Python function:{prompt}', 'stop': ['\nclass', '\ndef', '\nif', '\nprint'], 'model': 'text-davinci-002', 'max_tokens': 347, 'n': 36, 'temperature': 0.36865945026811975}
best result on tuning data {'expected_success': 0.7947565938469873, 'success': 0.8, 'total_cost': 1.0647, 'cost': 1.0647, 'inference_cost': 0.03736, 'training_iteration': 0, 'config': {'prompt': 2, 'stop': 0, 'subspace': {'model': 'text-davinci-002', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 36}}, 'config/prompt': 2, 'config/stop': 0, 'config/subspace': {'model': 'text-davinci-002', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 36}, 'experiment_tag': 'exp', 'time_total_s': 183.4394142627716}


### Make a request with the tuned config

We can apply the tuned config on the request for an example task:

In [14]:
responses = oai.Completion.create(context=tune_data[1], **config)
metric_results = success_metrics([response["message"]["content"] if config["model"] in oai.Completion.chat_models else response["text"] for response in responses["choices"]], **tune_data[1])
print("response on an example data instance:", responses)
print("metric_results on the example data instance:", metric_results)


StatementMeta(automl, 21, 21, Finished, Available)

response on an example data instance: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "    # your code here\n    return [abs(game[i] - guess[i]) for i in range(len(game))]"
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "logprobs": null,
      "text": "    # your code here\n    pass"
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "logprobs": null,
      "text": "    # your code here\n    return [abs(game[i]-guess[i]) for i in range(len(game))]"
    },
    {
      "finish_reason": "stop",
      "index": 3,
      "logprobs": null,
      "text": "    # your code here\n    return [abs(game[i] - guess[i]) for i in range(len(game))]"
    },
    {
      "finish_reason": "stop",
      "index": 4,
      "logprobs": null,
      "text": "    return [abs(game[i] - guess[i]) for i in range(len(game))]"
    },
    {
      "finish_reason": "stop",
      "index": 5,
      "logprobs": null,
      "te

### Evaluate the success rate on the test data

You can use flaml's `oai.Completion.test` to evaluate the performance of an entire dataset with the tuned config. The following code will take a while to evaluate all the 144 test data instances. The cost is about $7 if you uncomment it and run it.

In [15]:
result = oai.Completion.test(test_data, config, success_metrics)
print("performance on test data with the tuned config:", result)

StatementMeta(automl, 21, 22, Finished, Available)

performance on test data with the tuned config: {'expected_success': 0.7388060083964307, 'success': 0.7777777777777778, 'cost': 5.72968, 'inference_cost': 0.03978944444444445}


The result will vary with the inference budget and optimization budget.
